In [ ]:
import os
import numpy as np
import pandas as pd
from utils.plotting import init_profile_fig, add_profile

In [ ]:
dataset_dir = os.path.join('data','processed')
building_fname_pattern = 'ly_{id}-{year}.csv'

In [ ]:
building_id = 58
year = 2015

In [ ]:
# Load profile data.
original_profile = pd.read_csv(os.path.join(dataset_dir, building_fname_pattern.format(id=building_id, year=year)),
    usecols=['Equipment Electric Power [kWh]']
    )['Equipment Electric Power [kWh]'].to_numpy()

In [ ]:
print(np.mean(original_profile))
print(np.std(original_profile)/np.mean(original_profile))
print(np.max(original_profile))

In [ ]:
fig = init_profile_fig(y_titles={'y1':'Building energy (kWh)'})
fig = add_profile(fig, original_profile, name=f'Original')

In [ ]:
def scale_profile(profile, mean, peak):
    """Scale energy usage profile to have a given mean and peak.
    Method uses iterative scaling of peak and mean to achieve both.

    Args:
        profile (np.array): 1D array of energy usage profile.
        mean (float): Mean load to scale to.
        peak (float): Peak load to scale to.

    Returns:
        np.array: Scaled load profile.
    """

    p = profile.copy()
    p *= mean/np.mean(p)

    while np.abs(np.max(p) - peak) >= 0.1/100*peak:
        mu = np.mean(p)
        p = np.where(p > mu, (p-mu)*peak/np.max(p)+mu, p)
        p *= mean/np.mean(p)

    return p

In [ ]:
for i,(mean, peak) in enumerate([(50, 150), (100, 250), (150, 300)]):

    scaled_profile = scale_profile(original_profile, mean, peak)

    print(np.mean(scaled_profile))
    print(np.std(scaled_profile)/np.mean(scaled_profile))
    print(np.max(scaled_profile))

    fig = add_profile(fig, scaled_profile, name=f'Scaled {i}')
fig.write_html('test.html')
fig.show()